In [1]:
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import json
import re
from time import sleep
import random 
from datetime import datetime

from io import BytesIO
import zstandard as zstd

In [11]:
def write_data(data: pd.core.frame.DataFrame, path: str):
    """Функция для сохранения данных в виде Excel файлов.

    Аргументы:
        data (pd.core.frame.DataFrame): Датафрейм pandas
        path (str): Директория где хранятся данные.

    Результат:
        Сохранение файла Excel в директории path.
    """
    data_save = data.drop_duplicates().reset_index(drop=True)

    data_save.to_excel(path, index = False,
           engine='xlsxwriter', engine_kwargs={'options' : { 'strings_to_urls': False}} )    
    
def update_data(new_data: pd.core.frame.DataFrame, path_sales: str, path_supply: str, key: str):
    """Функция для обновления данных и фиксации продаж в Excel файлах.

    Аргументы:
        new_data (pd.core.frame.DataFrame): Датафрейм pandas
        path_sales (str): Директория где хранятся данные продаж.
        path_supply (str): Директория где хранятся данные актуальных объявлений.
        key (str): Признак, по которому проводится проверяется факт продажи. Часто адрес (url) страницы объявления.

    Результат:
        Актуальные и закрытые объявления проверяются и обновляется данные.
        Возвращается кол-во закрытых объявлений и новых объявлений.
    """
    
    date = f'{datetime.now().day}.{datetime.now().month}.{datetime.now().year}'
    
    "Если файла по данной директории нет, то создается файл с те ми же столбцами."
    try:
        sales = pd.read_excel(path_sales)
    except:
        write_data(pd.DataFrame(columns=list(new_data.columns) + ['sale_date']), path_sales)
        sales = pd.read_excel(path_sales)

    try:
        supply = pd.read_excel(path_supply)
    except:
        write_data(pd.DataFrame(columns=new_data.columns), path_supply)
        supply = pd.read_excel(path_supply)
    
    new_supply = new_data[~new_data[key].isin(supply[key])]
    sold_supply = supply[~supply[key].isin(new_data[key])]
    remaining_supply = supply[supply[key].isin(new_data[key])]
    
    if not new_supply.empty:
        
        save_supply = pd.concat([remaining_supply, new_supply])
        write_data(save_supply, path_supply)
    
    if not sold_supply.empty:
        sold_supply.loc[:, 'sale_date'] = date
        
        save_sales = pd.concat([sales, sold_supply])
        write_data(save_sales, path_sales)
        
        save_supply = pd.concat([remaining_supply, new_supply])
        write_data(save_supply, path_supply)
        
    return len(sold_supply), len(new_supply)